In [18]:
import tensorflow as tf
from GAN import GAN
#from WGAN import WGAN, GP_WGAN
import numpy as np
import os, gc
import matplotlib.pyplot as plt
import mnist
from skimage.transform import resize, rotate, warp, SimilarityTransform
%matplotlib inline

# Experiments with WGAN and mnist results
I tried different options with watered down but result images weren't good</br>
<ol>
    <li> WGAN GP takes a long time to train
    <li> WGAN does not create good images at short period of times
   
</ol>
## Forget about WGAN



In [3]:
def transform_image(img, code):
    if code == 0:
        return(img)
    
    if code == 1:
        ang = np.random.randint(low=-10, high=10)
        p = np.int(np.ceil(np.sin(np.deg2rad(np.abs(ang)))*img.shape[0]))
        img = rotate(img, ang)
        img[:, 0:p] = -1
        img[:,-p::] = -1
        img[0:p, :] = -1
        img[-p::, :] = -1
        
        return(img)
    
    
    if code == 2:
        sft = np.random.randint(-4, 4)
        tr = SimilarityTransform(scale=1, rotation=0, translation=[sft,sft])
        img = warp(img, tr)
        if sft>0:
            img[:,-sft::] = -1
            img[-sft::,:] = -1
        else:
            img[:,0:-sft] = -1
            img[0:-sft,:] = -1
        return(img)
        

In [4]:
mn = mnist.Mnist(path=".")
imgs, lbls, _ = mn.get_batch(60000)
imgs = (imgs - 0.5)*2


def get_batch(batch_size):
    global imgs
    rnd_idxs = np.random.choice(np.arange(imgs.shape[0]), batch_size)
    for idx in rnd_idxs:
        code = np.random.choice(np.arange(3), 1)[0]
        imgs[idx] = transform_image(imgs[idx], code=code)
    return (imgs[rnd_idxs])    
    
        

Images magic no: 2051,  No of images: 60000, Image rows: 28, Image cols: 28
Labels magic no: 2049, No of items: 60000


# DCGAN different settig mnist

In [34]:
import networkx as nx
g = nx.DiGraph()
nodes ={"sm":"sm", "ns_g":"ns_g", "ns_d_1":"ns_d_1", "ns_d_12":"ns_d_12", 
        "ns_d_123":"ns_d_123", "mbd_45":"mbd_45", "mbd_23":"mbd_23"}
g.add_edge(nodes["sm"], nodes["ns_g"], weight=0.4)
g.add_edge(nodes["sm"], nodes["ns_d_1"], weight=0.6)
g.add_edge(nodes["ns_g"], nodes["ns_d_1"], weight=1.)
g.add_edge(nodes["ns_d_1"], nodes["ns_d_12"], weight=0.4)
g.add_edge(nodes["ns_d_12"], nodes["ns_d_123"], weight=0.4)
g.add_edge(nodes["ns_d_1"], nodes["mbd_45"], weight=0.6)
g.add_edge(nodes["ns_d_12"], nodes["mbd_45"], weight=0.6)
g.add_edge(nodes["ns_d_123"], nodes["mbd_45"], weight=1.0)
g.add_edge(nodes["mbd_45"], nodes["mbd_23"], weight=1.0)
a_mat = nx.adjacency_matrix(g).toarray()

idxs = np.arange(7)
combinations = np.zeros([20, 7])
sel_n = list()
for i in range(combinations.shape[0]):
    combinations[i, 0] = 1
    row = 0
    for _ in range(7):
        if a_mat[row,:].sum()>0:
            row = np.random.choice(idxs, p=a_mat[row,:])
            combinations[i, row] = 1
        
#good
combinations


array([1., 1., 1., 1., 1., 1., 0.])

In [6]:
z_dim = 100

tf.reset_default_graph()
gan = GAN(verbose=True)
layers = list()

ns = tf.train.exponential_decay(0.1, tf.Variable(0, trainable=False), 1000, 0.95, staircase=True)

G_layers = [[1024, [4, 4], (1, 1), "valid", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [512, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [256, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [128, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [1, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 0, "th", None, 0]]

D_layers = [[128, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 0, 'lr', None, ns],
            [256, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, ns],
            [512, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, 0],
            [1024, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', [4, 5], 0],
            [1, [4, 4], (1, 1), 'valid', tf.random_normal_initializer(stddev=0.02), 0, 'l', None, 0]]

x = tf.placeholder(tf.float32, [None, 28, 28])
xr = tf.image.resize_images(tf.expand_dims(x, axis=3), (64, 64))

gargs = gan.build_graph(G_layers, D_layers, xr,
                       tf.placeholder(tf.float32, [None, 1, 1, z_dim]), True, 0.9, ["RMS"], 2e-4, True)

=====generator=====
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
-layer:0, no_of_filters:1024, k_size:[4, 4], strides:(1, 1), padding:valid, initializer:<tensorflow.python.ops.init_ops.RandomNormal object at 0x0000018093AD3198>, has_bn:1, act:r output shape:[None, 4, 4, 1024] mb:None noise:0-
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
-layer:1, no_of_filters:512, k_size:[4, 4], strides:(2, 2), padding:same, initializer:<tensorflow.python.ops.init_ops.RandomNormal object at 0x0000018093AD3358>, has_bn:1, act:r output shape:[None, 8, 8, 512] mb:None noise:0-
----------------------------------------------------------------------

In [ ]:
gargs = [g for g in gargs]
gargs[1] = x
gargs = tuple(gargs)

gan.train(60,  2000, 10, get_batch, "n", z_dim, True, "summary\\GAN_mnist_case_{}".format(i), "Model\\GAN_mnist_case_{}".format(i), None, None, gargs)